In [11]:
# selecting parameters
# number of convolutional filters to use
nb_filters = 64 

# filter size    
filter_rows = 1
filter_cols = 2 

# convolution kernel size
nb_conv = 3

# subsampling size 
subsample_rows = 1 
subsample_cols_a = 2    
subsample_cols_b = 1

# batch size 
batch_size = 50

# dimension    
latent_dim = 3
intermediate_dim = 128

# epsilon values 
mean=0.
epsilon_std = 1.0      

# dropout 
drop = 0

# epoch, iteration sizes  
nb_epoch = 1  

# saving models: start & end
nb_start = 0
nb_end = 400       
total_epochs = (nb_end-nb_start)*nb_epoch  
print "Total of %i epochs are set to run" % total_epochs  

# selecting path for array and data file (if saved in text or in pickle or compressed file)   
opt = 1  
# depending on 'opt' value choose path_data, path_data_array or path_data_dat    
# if 'opt=1' then choose path_data_array and path_data_dat, n_traj, f_traj, row_dim and col_dim     
# else for 'opt=2' or 'opt=3' choose path_data
if opt == 1:
    # number of trajectory files
    n_traj = 1#111
    # number of frames per trajectories
    f_traj = 10000
    # row and column dimension for each frame in the array file 
    row_dim_array = 28    
    col_dim_array = 28   
    # fraction of train, test and pred data separation 
    sep_train = 0.8     
    sep_test = 0.9    
    sep_pred = 1        
    # path of array and dat file 
    path_data_array = "/Users/odb/Documents/deeplearning_run/1FME-0/analysis/metadata/1FME-0_cont-mat.array"
    path_data_dat = '/Users/odb/Documents/deeplearning_run/1FME-0/analysis/metadata/1FME-0_cont-mat.dat'    
    print "option selected", opt

if opt != 1:    
    # fraction of train, test and pred data separation 
    sep_train = 0.8     
    sep_test = 0.9    
    sep_pred = 1        
    # path of data file 
    path_data = './X_110000.pkl.gz'  
    print "option selected", opt

#
print "parameter initialization finished"
#
###########################################################################################################
#
#!/usr/bin/python

import os, sys

path_1 = "./fig"
path_2 = "./imgs"
path_3 = "./hist"
path_4 = "./model"
if not os.path.exists(path_1):
    os.mkdir(path_1, 0755);
if not os.path.exists(path_2):
    os.mkdir(path_2, 0755);
if not os.path.exists(path_3):
    os.mkdir(path_3, 0755);
if not os.path.exists(path_4):
    os.mkdir(path_4, 0755);   
print "directories creation compledted or if already exist - then checked"    

    
import numpy as np
import matplotlib.pyplot as plt
import gzip
from six.moves import cPickle
import sys    

   
# reading dataset    
print "reding dataset starts"    
print "option chosen %i" % opt    

if opt == 1:  
    # option 1 
    # open separate array and data files stored in text file      
    # total number of frames        
    - 

if opt == 2:
    # option 2    
    # open compressed pickled data already separated in train, test and prediction data          
    with gzip.open(path_data, 'rb') as f3:    
        (x_train_raw, y_train_raw), (x_test_raw,y_test_raw), (x_pred_raw,y_pred_raw) = cPickle.load(f3)     

if opt == 3:
    # option 3 
    # open compressed pickled data not separated in train, test and prediction data     
    with gzip.open(path_data, 'rb') as f3:    
        (x_raw, y_raw) = cPickle.load(f3)  
    
    # determining size for training, testing & prediction data    
    sep_1 = x_raw.shape[0]*sep_train 
    sep_2 = x_raw.shape[0]*sep_test    
    sep_3 = x_raw.shape[0]*sep_pred   
    
    # assigning training set 
    x_train_raw = x_raw[:sep_1]
    y_train_raw = y_raw[:sep_1]  
    
    # assigning testing set 
    x_test_raw = x_raw[sep_1:sep_2]
    y_test_raw = y_raw[sep_1:sep_2] 
    
    # assigning prediction set 
    x_pred_raw = x_raw[sep_2:sep_3]
    y_pred_raw = y_raw[sep_2:sep_3]  


print 'printing shape of dataset:'
print 'train set:', np.shape(x_train_raw), np.shape(y_train_raw) 
print 'test set:', np.shape(x_test_raw), np.shape(y_test_raw) 
print 'pred set:', np.shape(x_pred_raw), np.shape(y_pred_raw)
print "data read and shape is now written"


# model building and compiling
print "now building model and compiling"
import keras
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Dropout
from keras.layers import Convolution2D, Deconvolution2D
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist   
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.callbacks import Callback



# normalizing input image matrix 
a = np.amax(x_train_raw)
b = np.amax(x_test_raw)
c = np.amax(x_pred_raw)
x_train = x_train_raw.astype('float32') / a
x_test = x_test_raw.astype('float32') / b
x_pred = x_pred_raw.astype('float32') / c

# input image dimensions   
print "reshaping image dimension"
img_rows = x_train_raw.shape[0]/len(x_train_raw)    
img_cols = x_train_raw.shape[1]
img_rows, img_cols, img_chns = img_rows, img_cols, 1

if K.image_dim_ordering() == 'th':
    original_img_size = (img_chns, img_rows, img_cols)
else:
    original_img_size = (img_rows, img_cols, img_chns)

x_train = x_train.reshape((x_train.shape[0],) + original_img_size)
x_test = x_test.reshape((x_test.shape[0],) + original_img_size)
x_pred = x_pred.reshape((x_pred.shape[0],) + original_img_size)  

x = Input(batch_shape=(batch_size,) + original_img_size)


# defining convolutional variational autoencoders        
'''building variational autoencoder with Keras and deconvolution layers.
Reference: Variational Autoencoder: "Auto-Encoding Variational Bayes" (https://arxiv.org/abs/1312.6114)
'''
def vae_conv():    
    conv_1 = Convolution2D(img_chns, filter_rows, filter_cols, border_mode='same', activation='relu')(x)
    conv_2 = Convolution2D(nb_filters, filter_rows, filter_cols,
                       border_mode='same', activation='relu',
                       subsample=(subsample_rows, subsample_cols_a))(conv_1)
    conv_3 = Convolution2D(nb_filters, nb_conv, nb_conv,
                       border_mode='same', activation='relu',
                       subsample=(subsample_rows, subsample_cols_b))(conv_2)
    conv_4 = Convolution2D(nb_filters, nb_conv, nb_conv,
                       border_mode='same', activation='relu',
                       subsample=(subsample_rows, subsample_cols_b))(conv_3)
    flat = Flatten()(conv_4)
    flat_d = Dropout(drop)(flat)
    hidden = Dense(intermediate_dim, activation='relu')(flat_d)

    z_mean = Dense(latent_dim)(hidden)
    z_log_var = Dense(latent_dim)(hidden)        

    # sampling on latent space 
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=mean, std=epsilon_std)
        return z_mean + K.exp(z_log_var) * epsilon

    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    # we instantiate these layers separately so as to reuse them later
    decoder_hid = Dense(intermediate_dim, activation='relu')
    decoder_upsample = Dense(nb_filters * img_rows * (img_cols/2), activation='relu')

    if K.image_dim_ordering() == 'th':
        output_shape = (batch_size, nb_filters, img_rows, (img_cols/2))
    else:
        output_shape = (batch_size, img_rows, (img_cols/2), nb_filters)

    decoder_reshape = Reshape(output_shape[1:])
    decoder_deconv_1 = Deconvolution2D(nb_filters, nb_conv, nb_conv,
                                   output_shape,
                                   border_mode='same',
                                   subsample=(subsample_rows, subsample_cols_b),
                                   activation='relu')
    decoder_deconv_2 = Deconvolution2D(nb_filters, nb_conv, nb_conv,
                                   output_shape,
                                   border_mode='same',
                                   subsample=(subsample_rows, subsample_cols_b),
                                   activation='relu')
    if K.image_dim_ordering() == 'th':
        output_shape = (batch_size, nb_filters, img_rows, (img_cols+1))
    else:
        output_shape = (batch_size, img_rows, (img_cols+1), nb_filters)
    decoder_deconv_3_upsamp = Deconvolution2D(nb_filters, filter_rows, filter_cols,
                                          output_shape,
                                          border_mode='valid',
                                          subsample=(subsample_rows, subsample_cols_a),
                                          activation='relu')
    decoder_mean_squash = Convolution2D(img_chns, filter_rows, filter_cols,
                                    border_mode='valid',
                                    activation='sigmoid')

    hid_decoded = decoder_hid(z)
    up_decoded = decoder_upsample(hid_decoded)
    reshape_decoded = decoder_reshape(up_decoded)
    deconv_1_decoded = decoder_deconv_1(reshape_decoded)
    deconv_2_decoded = decoder_deconv_2(deconv_1_decoded)
    x_decoded_relu = decoder_deconv_3_upsamp(deconv_2_decoded)
    x_decoded_mean_squash = decoder_mean_squash(x_decoded_relu)  
    
    return z_mean, z_log_var, x_decoded_mean_squash

def vae_loss(x, x_decoded_mean):
    vae_conv_return = vae_conv()
    z_mean, z_log_var, x_decoded_mean_squash = vae_conv_return
    # NOTE: binary_crossentropy expects a batch_size by dim
    # for x and x_decoded_mean, so we MUST flatten these!
    x = K.flatten(x)
    x_decoded_mean = K.flatten(x_decoded_mean)
    xent_loss = img_rows * img_cols * objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    
    return xent_loss + kl_loss

# model to run 
vae_conv_return = vae_conv()
z_mean, z_log_var, x_decoded_mean_squash = vae_conv_return
vae = Model(x, x_decoded_mean_squash)   
print "model selection finished"


# Optimizers selection  
rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
#adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)  
print "optimizers selection finished"

# compiling model 
vae.compile(optimizer=rms, loss=vae_loss)
print "model compilation finished"

# model summary 
print "model summary:"  
vae.summary()     
print "model building and compilation finished now"




# saving history 
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

history = LossHistory()

# 1) fitting model with training dataset, saving model & loading for next fitting    
# 2) saving loss values 
for i in range (nb_start, nb_end):    
    if i == 0:        
        print('skipping - no previous saved file to load')
        
    else:        
        vae.load_weights('./model/model_%i' % i)       
                
    vae.fit(x_train, x_train,
        shuffle=True,
        nb_epoch=nb_epoch,
        batch_size=batch_size,
#        verbose=2,
        validation_data=(x_test, x_test),
        callbacks=[history])       
    
    vae.save_weights('./model/model_%i' % (i+1))
    print('completed %i epochs' % ((i+1)*nb_epoch)) 
    
    np.savetxt('./hist/history.losses_%i' %(i+1), history.losses, delimiter=',')
    np.savetxt('./hist/history.val_losses_%i' %(i+1), history.val_losses, delimiter=',')
    print "saved history files"    




# finishing the job 
print "the job is finsihed. You can find the model and the history files in the respective folders."


Total of 400 epochs are set to run
option selected 1
parameter initialization finished
directories creation compledted or if already exist - then checked
reding dataset starts
option chosen 1
frames read: 0
initial matrix array dimension: (280000, 28)
final matrix array dimension: (10000, 28, 28)
dat dimension: (1110000, 3)
printing shape of dataset:
train set: (8000, 784) (8000, 3)
test set: (1000, 784) (1000, 3)
pred set: (1000, 784) (1000, 3)
data read and shape is now written
now building model and compiling
reshaping image dimension


/Users/odb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:150: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/odb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:151: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/odb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:154: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/odb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:155: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/odb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:158: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/odb/anaconda/lib/python2.7/site-packages/ipykern

model selection finished
optimizers selection finished
model compilation finished
model summary:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (50, 1, 1, 784)       0                                            
____________________________________________________________________________________________________
convolution2d_21 (Convolution2D) (50, 1, 1, 784)       3           input_3[0][0]                    
____________________________________________________________________________________________________
convolution2d_22 (Convolution2D) (50, 64, 1, 392)      192         convolution2d_21[0][0]           
____________________________________________________________________________________________________
convolution2d_23 (Convolution2D) (50, 64, 1, 392)      36928       convolution2d_22[0][0]      

KeyboardInterrupt: 